# NHITS Model

## Data Manipulation

In [2]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [3]:
final_merged_df = pd.read_csv('../data/final_merged_data.csv', parse_dates=['Local Time'])

# Print preliminary data
print(f"Loaded {len(final_merged_df):,} records")
print(f"Number of unique rides: {final_merged_df['Ride'].nunique()}")

# Print a list of all unique ride names along with the number of records for each ride.
# Using value_counts() which is more reliable than manual counting
print(f"\nUnique rides:")
ride_counts = final_merged_df['Ride'].value_counts().sort_index()
for ride, count in ride_counts.items():
    print(f"  {ride}: {count:,} records")

/var/folders/pp/vwp6cpmd6xd962fpyb8gjv5w0000gn/T/ipykernel_88543/1020655066.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  final_merged_df = pd.read_csv('../data/final_merged_data.csv', parse_dates=['Local Time'])


Loaded 589,985 records
Number of unique rides: 42

Unique rides:
  Animation Academy: 18,021 records
  Games of Pixar Pier: 16,166 records
  Golden Zephyr: 18,021 records
  Goofy's Sky School: 18,021 records
  Grizzly River Run: 18,021 records
  Guardians of the Galaxy - Mission: BREAKOUT!: 18,021 records
  Guardians of the Galaxy - Monsters After Dark: 9,754 records
  Incredicoaster: 18,021 records
  Incredicoaster Single Rider: 18,021 records
  Inside Out Emotional Whirlwind: 18,021 records
  Jessie's Critter Carousel: 18,021 records
  Jumpin' Jellyfish: 18,021 records
  Luigi's Honkin' Haul-O-Ween: 1,516 records
  Luigi's Joy to the Whirl: 3,087 records
  Luigi's Rollickin' Roadsters: 11,564 records
  Mater's Graveyard JamBOOree: 3,344 records
  Mater's Jingle Jamboree: 3,087 records
  Mater's Junkyard Jamboree: 11,590 records
  Mickey's PhilharMagic: 18,021 records
  Monsters, Inc. Mike & Sulley to the Rescue!: 18,021 records
  Pixar Pal-A-Round - Swinging: 18,021 records
  Pixar P

In [4]:
# Dropping rides that are redundant/stupid
rides_to_drop = [
    'Animation Academy',
    'Games of Pixar Pier',
    'Guardians of the Galaxy - Monsters After Dark',
    'Luigi\'s Honkin\' Haul-O-Ween',
    'Luigi\'s Joy to the Whirl',
    'Mater\'s Graveyard JamBOOree',
    'Mater\'s Jingle Jamboree',
    'Red Car Trolley',
    'Redwood Creek Challenge Trail',
    'The Bakery Tour',
    'Sorcerer\'s Workshop',
    'Soarin\' Over California',
    'Rogers: The Musical', 
    'Walt Disney Imagineering Blue Sky Cellar', 
    'Turtle Talk with Crush', 
    'Mickey\'s PhilharMagic',
    'World of Color – ONE',
    'World of Color - Season of Light',
    'Villains Grove at Oogie Boogie Bash',
    'Silly Symphony Swings Single Rider',
    'Incredicoaster Single Rider',
    'Radiator Springs Racers Single Rider'
]

print(f"Rides before filtering: {final_merged_df['Ride'].nunique()}")
print(f"Total records before filtering: {len(final_merged_df):,}")

# Filter out the specified rides
final_merged_df = final_merged_df[~final_merged_df['Ride'].isin(rides_to_drop)]

print(f"Rides after filtering: {final_merged_df['Ride'].nunique()}")
print(f"Total records after filtering: {len(final_merged_df):,}")

print(f"\nDropped rides:")
for ride in rides_to_drop:
    print(f"  - {ride}")

Rides before filtering: 42
Total records before filtering: 589,985
Rides after filtering: 20
Total records after filtering: 344,631

Dropped rides:
  - Animation Academy
  - Games of Pixar Pier
  - Guardians of the Galaxy - Monsters After Dark
  - Luigi's Honkin' Haul-O-Ween
  - Luigi's Joy to the Whirl
  - Mater's Graveyard JamBOOree
  - Mater's Jingle Jamboree
  - Red Car Trolley
  - Redwood Creek Challenge Trail
  - The Bakery Tour
  - Sorcerer's Workshop
  - Soarin' Over California
  - Rogers: The Musical
  - Walt Disney Imagineering Blue Sky Cellar
  - Turtle Talk with Crush
  - Mickey's PhilharMagic
  - World of Color – ONE
  - World of Color - Season of Light
  - Villains Grove at Oogie Boogie Bash
  - Silly Symphony Swings Single Rider
  - Incredicoaster Single Rider
  - Radiator Springs Racers Single Rider


In [5]:
# Split the main dataset into separate, time-sorted DataFrames for each
# ride and store them in a dictionary for easy per-ride analysis.
ride_dataframes = {}

for ride in final_merged_df['Ride'].unique():
    ride_df = final_merged_df[final_merged_df['Ride'] == ride].copy()
    ride_df = ride_df.sort_values('Local Time').reset_index(drop=True)
    ride_dataframes[ride] = ride_df

for ride, df in ride_dataframes.items():
    
    # Feature Engineering (make raw data easier to use)
    df['hour'] = df['Local Time'].dt.hour
    df['dow'] = df['Local Time'].dt.dayofweek
    df['month'] = df['Local Time'].dt.month
    df['weekend'] = df['dow'].isin([5,6]).astype(int)

## NHITS

In [ ]:
# Training NHITS

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS

all_predictions = []
ride_mae_results = {}

for ride_name, ride_df in ride_dataframes.items():
    print(f"\n{'='*60}")
    print(f"Training NHITS for: {ride_name}")
    print(f"{'='*60}")

    df = ride_df.copy()

    # Rename to expected column names
    df = df.rename(columns={
        'Local Time': 'ds',
        'Wait Time': 'y',
        'holiday_flag': 'holiday',
        'apparent_temperature (°F)': 'temp',
        'precipitation (mm)': 'prec'
    })

    df['unique_id'] = ride_name
    df = df[['unique_id', 'ds', 'y', 'tier', 'holiday', 'temp', 'prec', 'hour', 'dow', 'month', 'weekend']].reset_index(drop=True)
    exog_cols = ['tier', 'holiday', 'temp', 'prec', 'hour', 'dow', 'month', 'weekend']

    df['ds'] = pd.to_datetime(df['ds'])
    df['ds'] = df['ds'].dt.floor('15min')

    # Sort by ride and time
    df = df.sort_values(['unique_id','ds'])

    HOLDOUT_FRACTION = 0.3
    split_idx = int(len(df) * (1 - HOLDOUT_FRACTION))
    train = df.iloc[:split_idx].copy()
    test = df.iloc[split_idx:].copy()

    try: 
        # NHITS model
        model = NHITS(
            input_size=96,
            h=15,
            n_blocks=[2, 2, 2],
            mlp_units=[[256, 256],
                    [256, 256],
                    [256, 256]],
            n_pool_kernel_size=[2, 4, 8],
            batch_size=32,
            learning_rate=1e-3,
            max_steps=30000,
            hist_exog_list=exog_cols
        )

        nf = NeuralForecast(models=[model], freq='15min')
        nf.fit(train)

        fcst = nf.predict()

        merged_data = test.merge(fcst, on=['unique_id', 'ds'], how='inner')
        
        if len(merged_data) > 0:
            mae = mean_absolute_error(merged_data['y'], merged_data['NHITS'])
            ride_mae_results[ride_name] = mae
            print(f"MAE: {mae:.2f} minutes")
        else:
            ride_mae_results[ride_name] = None
            print("No matching predictions")
    except Exception as e:
        print(f"Error: {e}")
        ride_mae_results[ride_name] = None


results_df = pd.DataFrame({
    'Ride': list(ride_mae_results.keys()),
    'Model': 'NHITS',
    'MAE': list(ride_mae_results.values())
})
results_df.to_csv('../data/results/nhits_results.csv', index=False)
print(f"\nSaved results to '../data/results/nhits_results.csv'")
print(results_df)

Seed set to 1



Training NHITS for: Guardians of the Galaxy - Mission: BREAKOUT!


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 1.7 M  | train
-------------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.935     Total estimated model params size (MB)
64        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=30000` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

MAE: 0.10 minutes

Saved results to '../data/results/nhits_results.csv'
                                                 Ride  Model        MAE
0        Guardians of the Galaxy - Mission: BREAKOUT!  NHITS  15.839087
1                                       Golden Zephyr  NHITS   5.541721
2                                  Goofy's Sky School  NHITS   8.282196
3                               Silly Symphony Swings  NHITS   2.700727
4     The Little Mermaid - Ariel's Undersea Adventure  NHITS   6.445615
5                                      Incredicoaster  NHITS  14.438413
6                      Inside Out Emotional Whirlwind  NHITS   8.517039
7                           Jessie's Critter Carousel  NHITS   8.571496
8                                   Jumpin' Jellyfish  NHITS   3.580470
9                        Pixar Pal-A-Round - Swinging  NHITS   6.436307
10                   Pixar Pal-A-Round – Non-Swinging  NHITS   7.771111
11                            Toy Story Midway Mania!  NHITS   9